In [75]:
from simulated_annealing import *

In [16]:
def eval_solution(solution, input_data):
    score = 0
    c1, c2, c3, c4, c5 = demand_constraint, \
                         day_off_constraint, \
                         length_work_blocks_constraint, \
                         forbidden_constraint2, \
                         forbidden_constraint3
    if c1(solution, input_data): score += 50
    if c2(solution, input_data): score += 15
    if c3(solution, input_data): score += 15
    if c4(solution, input_data): score += 10
    if c5(solution, input_data): score += 10
    
    return score

In [17]:
def exp_probability(solution, neighbor, T, input_data):
    return math.exp((eval_solution(neighbor, input_data) - eval_solution(solution, input_data))/T)

In [18]:
def simulated_annealing(input_data, T_max, r, termination_condition, halting_condition):
    tc = 0
    hc = 0
    score = 100
    T = T_max
    
    solution = generate_random_solution(input_data)
    
    while hc < halting_condition:
        while tc < termination_condition:
            if eval_solution(solution, input_data) == 100: 
                return solution
            
            n1 = update_forbidden_constraint3(solution, input_data)
            n2 = update_forbidden_constraint3_2(solution, input_data)
            n3 = update_forbidden_constraint2(solution, input_data)
            n4 = update_forbidden_constraint2_2(solution, input_data)
            n5 = update_length_work_blocks_constraint(solution, input_data)
            n6 = update_day_off_constraint(solution, input_data)
            n7 = update_demand_constraint(solution, input_data)

            neighborhood = [n1, n2, n3, n4, n5, n6, n7]

            neighbor = random.choice(neighborhood)

            if eval_solution(solution, input_data) < eval_solution(neighbor, input_data): solution = neighbor
            elif random.uniform(0, 1) < exp_probability(solution, neighbor, T, input_data): solution = neighbor
            
            tc += 1
            
        T *= r    
        hc += 1
        
    return "Not satified in the given time"

In [31]:
def eval_SA_params(input_data, T_max, tc_max, hc_max, time_limit):
    print(f"Running SA on {input_data['filename']}")
    params = []
    start_time_limit = time.time()
    for T in range(1, T_max):
        end_time_limit = time.time()
        if end_time_limit - start_time_limit > time_limit:
            break
        for hc in range(1, hc_max):
            end_time_limit = time.time()
            if end_time_limit - start_time_limit > time_limit:
                break
            for tc in range(1, tc_max):
                end_time_limit = time.time()
                if end_time_limit - start_time_limit > time_limit:
                    print(f'Exceeded time limit of {time_limit} seconds.')
                    break
                start = time.time()
                solution = simulated_annealing(input_data, T, 0.99, tc, hc)
                end = time.time()
                if solution != "Not satified in the given time":
                    print(f'Parameters solution T={T}\tr={0.99}\ttermination_condition={tc}\thalting_condition={hc}\tRuntime {end - start}')
                    params.append((T, r/100, tc, hc, end - start))
    return params

In [20]:
def get_examples():
    RWS_INSTANCES = 'rws_instances/'
    EXAMPLE = 'Example'
    filenames = []

    for i in range(1,21):
        filenames.append(RWS_INSTANCES + EXAMPLE + str(i) + '.txt')
    return filenames

In [ ]:
def run_SA_eval_on_examples():
    T = 100
    tc = 100
    hc = 100
    time_limit = 100
    params = {}
    for example in get_examples():
        try:
            input_data = read_data(example)
            input_data['filename'] = example
            params[example] = eval_SA_params(input_data, T, tc, hc, time_limit)
        except Exception as e:
            print(e)
    return params

In [22]:
T = 100
r = 0.99
tc = 1000
hc = 1000
solution = simulated_annealing(read_data(get_examples()[3]), T, r/100, tc, hc)
solution

[['D', 'D', 'D', 'D', 'D', '-', 'N'],
 ['-', 'D', 'D', 'D', 'D', 'D', 'N'],
 ['-', 'D', 'D', 'D', 'A', 'N', 'N'],
 ['D', 'D', 'D', 'D', '-', 'N', 'N'],
 ['-', 'D', 'D', 'D', 'D', 'D', 'N'],
 ['D', '-', 'A', 'A', 'A', 'A', 'A'],
 ['A', 'A', 'A', 'A', 'A', 'A', '-'],
 ['D', 'D', 'D', '-', 'D', 'D', 'D'],
 ['D', 'D', 'N', 'N', 'N', 'N', '-'],
 ['-', 'D', '-', 'D', 'A', 'A', 'N'],
 ['-', 'A', 'A', 'A', 'A', 'A', 'N'],
 ['D', 'D', 'D', 'D', '-', 'D', 'N'],
 ['D', 'D', 'D', '-', 'D', 'D', '-']]

In [44]:
def run_SA_on_examples():
    T = 100
    r = 0.99
    tc = 1000
    hc = 1000
    solutions = {}
    for example in get_examples():
        try:
            solution = simulated_annealing(read_data(example), T, r/100, tc, hc)
            print(example)
            pprint(solution)
            solutions[example] = solution
        except Exception as e:
            print(e)

In [52]:
def test_reading_input():
    for example in get_examples():
        try:
            print(example)
            pprint(read_data(example))
        except Exception as e:
            print('------------------------------------------------------------------------------------')
            print(e)
            print('------------------------------------------------------------------------------------')

In [60]:
def test_constraints():
    for example in get_examples():
        try:
            print(example)
            solution = generate_random_solution(read_data(example))
            pprint(eval_solution(solution, read_data(example)))
        except Exception as e:
            print('------------------------------------------------------------------------------------')
            print(e)
            print('------------------------------------------------------------------------------------')

In [73]:
def test_SA():
    T_max = 100
    r = 0.99
    termination_condition = 1000
    halting_condition = 1000000
    for example in get_examples():
        try:
            print(example)
            input_data = read_data(example)
            solution = generate_random_solution(input_data)
            solution = simulated_annealing(input_data, T_max, r, termination_condition, halting_condition)
            pprint(eval_solution(solution, input_data))
            print('passed')
        except Exception as e:
            print('------------------------------------------------------------------------------------')
            print('Not passed')
            print(e)
            print('------------------------------------------------------------------------------------')

In [74]:
test_SA()

rws_instances/Example1.txt
------------------------------------------------------------------------------------
Not passed
string index out of range
------------------------------------------------------------------------------------
rws_instances/Example2.txt
------------------------------------------------------------------------------------
Not passed
string index out of range
------------------------------------------------------------------------------------
rws_instances/Example3.txt
------------------------------------------------------------------------------------
Not passed
string index out of range
------------------------------------------------------------------------------------
rws_instances/Example4.txt
100
passed
rws_instances/Example5.txt
------------------------------------------------------------------------------------
Not passed
string index out of range
------------------------------------------------------------------------------------
rws_instances/Example6.txt

In [45]:
solutions = run_SA_on_examples()

rws_instances/Example1.txt
'Not satified in the given time'
rws_instances/Example2.txt
'Not satified in the given time'
rws_instances/Example3.txt
'Not satified in the given time'
rws_instances/Example4.txt
[['-', '-', 'D', '-', 'A', 'A', 'A'],
 ['D', 'D', '-', 'D', 'A', 'N', 'N'],
 ['-', 'D', 'A', 'A', 'A', 'A', '-'],
 ['-', 'D', 'D', 'D', 'A', 'N', '-'],
 ['D', 'D', 'D', 'D', 'D', 'D', '-'],
 ['D', 'D', '-', 'D', 'D', 'N', 'N'],
 ['D', 'A', 'A', 'A', '-', 'A', 'N'],
 ['D', 'D', '-', 'D', 'D', 'A', 'N'],
 ['D', 'D', 'D', 'D', '-', 'N', 'N'],
 ['D', 'D', 'D', '-', 'D', 'N', 'N'],
 ['D', '-', 'D', 'D', 'D', 'D', 'N'],
 ['A', 'A', 'A', 'A', '-', 'A', 'N'],
 ['D', 'D', 'D', 'D', '-', 'D', 'N']]
rws_instances/Example5.txt
'Not satified in the given time'
rws_instances/Example6.txt
'Not satified in the given time'
rws_instances/Example7.txt
'Not satified in the given time'
rws_instances/Example8.txt
'Not satified in the given time'
list index out of range
rws_instances/Example10.txt
'Not sa